ITERATIVE FUNCTION

#We have the NU as the input, and it returns Y as the output. So we want to iterate that but changing the NU

In [1]:
import os
import numpy as np
from DMF_model import DMF_sim, DMF_parameters

In [2]:
# Create folder to store the results
folder = "COMBIS"
folder_path = os.path.join(os.getcwd(), folder)
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

In [17]:
# DMF parameters
P = DMF_parameters({})
P['sigma'] = 0.02  # no noise

# Simulation parameters   
t_sim = 3 # simulation time
t_start = 0.5 # stimulation start time
t_stop = 2.5 # stimulation stop time
dt = P['dt']     # integration time step
P['T'] = t_sim

transient_end = int(0.1/dt) # end of initial transient

# layer specific external input
stim_start = int(t_start/dt)    # start of stimulation
stim_end   = int(t_stop/dt)    # end of stimulation
sim_steps = int(t_sim/dt)       # number of simulation steps

stim_index = range(stim_start,stim_end) # indices of stimulation
input_template = np.zeros(sim_steps) # input current
input_template[stim_index] = 1.0 # input current during stimulation (later just multiply with the specific input intensity)

# Define the possible values for each input intensity.
valuesI = [50, 100, 150, 200, 250, 300]
valuesLayer = range(8)

# Create an array of all possible combinations for each input
input_combinations = np.array(np.meshgrid(valuesI, valuesI)).T.reshape(-1, 2)
layer_combinations = np.array(np.meshgrid(valuesLayer, valuesLayer)).T.reshape(-1, 2)

# repeat each input combination for each layer combination
input_combinations = np.repeat(input_combinations, len(layer_combinations), axis=0)
layer_combinations = np.tile(layer_combinations, (len(valuesI)**2, 1))

# repeat each input combination and layer combination 5 times
input_combinations = np.repeat(input_combinations, 5, axis=0)
layer_combinations = np.repeat(layer_combinations, 5, axis=0)

# Create random input combinations centered around the original input combinations
input_combinations = input_combinations + np.random.uniform(-50, 49, input_combinations.shape)

# save input and layer combinations
np.save(os.path.join(folder_path, "input_combinations.npy"), input_combinations)
np.save(os.path.join(folder_path, "layer_combinations.npy"), layer_combinations)

In [50]:
# The loop

baseline_duration_added = 10
baseline_duration_steps = int(baseline_duration_added/dt)
baseline = np.load("baseline.npy")
baseline = baseline[:baseline_duration_steps,:]
Y_baseline = np.tile(baseline, (10,1,1))

duration = sim_steps - transient_end 
total_number_combinations = len(input_combinations)

# divide the total number of combinations into 10 batches and save each batch separately
batch_size = int(total_number_combinations/10)

Y = np.zeros((duration, 8, batch_size))
batch_counter = 0
for total_counter, (current, layer) in enumerate(zip(input_combinations, layer_combinations)):
    U = np.zeros((sim_steps, 8))
    U[:, layer[0]] = input_template * current[0]
    U[:, layer[1]] = input_template * current[1]

    # Run the simulation
    I, _, _ = DMF_sim(U, P)
    Y[:, :, batch_counter] = I[transient_end:, :]
    
    batch_counter += 1
    if (total_counter + 1) % batch_size == 0:
        Y3 = np.append(Y, Y_baseline)
        # Save the results and include the batch number in the file name with leading zeros
        np.save(os.path.join(folder_path, f'Y_{batch_counter:02d}.npy'), Y3)
        break
        Y = np.zeros((duration, 8, batch_size))
        batch_counter = 0
        
        

In [53]:
baseline_duration_added = 10
baseline_duration_steps = int(baseline_duration_added/dt)
baseline = np.load("baseline.npy")
baseline = baseline[:baseline_duration_steps,:]
baseline2 = np.repeat(baseline, 10, axis=1)
duration = sim_steps - transient_end 
batch_size = 10
Y2 = np.tile(baseline, (10,1,1))
    
#print(Y)
Y.shape


(29000, 8, 3)